In [1]:
import pandas as pd
import configparser
import psycopg2
import boto3
import json
import boto3
import time
from threading import Timer
import logging
%load_ext sql

logger = logging.getLogger() #(__name__)
logging.basicConfig(format="[%(levelname)s] [%(asctime)s] %(message)s")
logger.setLevel(logging.INFO)

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_IAM_ROLE_NAME      = config.get("DWH","DWH_IAM_ROLE_NAME")

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

sampleDbBucket =  s3.Bucket("udacity-dend")
'''
Prefix="data-pipelines"
Prefix="log-data"
Prefix="pagila"
Prefix="song-data"
Prefix="log_json_path"
'''
for obj in sampleDbBucket.objects.filter(Prefix="song-data").limit(5):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json')


In [37]:
# itemInSession 	length 	registration 	sessionId 	status 	ts  originaly the only ones of numeric type
dfs_ = []
for obj in sampleDbBucket.objects.filter(Prefix="log-data"):
    tmp_json = obj.get()['Body'].read().decode('utf-8')
    edata = pd.read_json(tmp_json, lines=True)
    dfs_.append(edata)
df_events_ = pd.concat(dfs_, ignore_index=True) # (8056, 18)
df_events = df_events_.loc[df_events_.page == "NextSong"].reset_index(drop=True) # (6820, 18) -> (6820, 25)

df_events["start_time"] = df_events["ts"]
df_events["hour"] = df_events.start_time.apply(lambda x: pd.Timestamp(x, unit='ms').hour)
df_events["day"] = df_events.start_time.apply(lambda x: pd.Timestamp(x, unit='ms').day)
df_events["week"] = df_events.start_time.apply(lambda x: pd.Timestamp(x, unit='ms').week)
df_events["month"] = df_events.start_time.apply(lambda x: pd.Timestamp(x, unit='ms').month)
df_events["year"] = df_events.start_time.apply(lambda x: pd.Timestamp(x, unit='ms').year)
df_events["weekday"] = df_events.start_time.apply(lambda x: pd.Timestamp(x, unit='ms').weekday()) # Monday == 0 … Sunday == 6

In [40]:
df_events.shape

(6820, 25)

In [ ]:
# too many objects ... one file per song ...
# don't run this
dfs = []
for sobj in sampleDbBucket.objects.filter(Prefix="song-data"):
    stmp_json = sobj.get()['Body'].read().decode('utf-8')
    sdata = pd.read_json(stmp_json, lines=True)
    dfs.append(sdata)
df_songs_ = pd.concat(dfs, ignore_index=True) # (8056, 18)
#df_songs = df_songs_.loc[df_songs_.page == "NextSong"].reset_index(drop=True) # (6820, 18)

In [4]:
len(dfs)

3354

In [28]:
import pandas as pd
'''
['artist_id', text
 'artist_latitude', numeric,
 'artist_location', test
 'artist_longitude', numeric,
 'artist_name', text
 'duration', numeric,
 'num_songs', numeric,
 'song_id', text
 'title', text
 'year'] numeric,
'''
content_object = s3.Object('udacity-dend', 'song-data/A/A/A/TRAAACN128F9355673.json')
file_content = content_object.get()['Body'].read().decode('utf-8')
data = pd.read_json(file_content, lines=True)

In [29]:
content_object

s3.Object(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json')

In [30]:
data.columns.tolist()

['artist_id',
 'artist_latitude',
 'artist_location',
 'artist_longitude',
 'artist_name',
 'duration',
 'num_songs',
 'song_id',
 'title',
 'year']

In [33]:
data.title[0]

'Hit Da Scene'